## Naive Bayes on Political Text - Mendelina Lopez

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [2]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from string import punctuation
from nltk.corpus import stopwords
import pandas as pd
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from ast import literal_eval

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

# Some punctuation variations
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

# Stopwords
sw = stopwords.words("english")


def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    return(word_tokenize(text))

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

C:\Users\mendi\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [8]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

# I would like to see if there are other tables
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(convention_cur.fetchall())

[('conventions',)]


In [9]:
# I would like to view the columns within the table
data = convention_cur.execute('''SELECT * FROM conventions''')
for column in data.description:
    print(column[0])

party
night
speaker
speaker_count
time
text
text_len
file


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [10]:
my_pipeline = [str.lower, remove_punctuation, tokenize]
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            WHERE speaker != "Unknown"
                            ''')

for row in query_results :
    # store the results in convention_data
    text = " ".join(prepare(row[0],pipeline=my_pipeline))
    if text :
        convention_data.append([" ".join(prepare(row[0],pipeline=my_pipeline)),row[1]])

Let's look at some random entries and see if they look right. 

In [27]:
random.choices(convention_data,k=10)

[['it will be led by joe biden and it ’ s aimed at ending the disease that killed the vice president ’ s son and 600000 other americans every year',
  'Democratic'],
 ['the first thing a president needs to do is find out what the facts are this president doesn ’ t care about facts tom c 015523 biden cares about the safety and welfare of american servicemen and women',
  'Democratic'],
 ['as the next', 'Democratic'],
 ['and finally joe biden will officially accept the nomination for president i ’ ll be tuning in i hope you will too thank you for letting me be a part of your night',
  'Democratic'],
 ['but what are the consequences when only one side of the story gets out or when only one viewpoint is acceptable for our education system it meant sacrificing civil debate by creating an atmosphere where students with contrary opinions are too afraid to speak many students find themselves suppressing their beliefs to fit into what the acceptable groupthink is in short our nation suffers by 

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [28]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2441 as features in the model.


In [31]:
def conv_features(text,fw = feature_words) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    text_tokens = text.split()
    text_feat_words = [ch for ch in text_tokens if ch in fw]
    
    
    ret_dict = dict.fromkeys(text_feat_words, True)
    
    return(ret_dict)

In [32]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

AssertionError: 

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [33]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [34]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [35]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.448


In [36]:
classifier.show_most_informative_features(25)

Most Informative Features
                 radical = True           Republ : Democr =     38.0 : 1.0
                   media = True           Republ : Democr =     38.0 : 1.0
                   votes = True           Democr : Republ =     22.8 : 1.0
             enforcement = True           Republ : Democr =     18.4 : 1.0
                   crime = True           Republ : Democr =     17.2 : 1.0
                   china = True           Republ : Democr =     16.3 : 1.0
                 destroy = True           Republ : Democr =     16.2 : 1.0
                freedoms = True           Republ : Democr =     16.2 : 1.0
                  defund = True           Republ : Democr =     14.1 : 1.0
                  earned = True           Republ : Democr =     14.1 : 1.0
                  lowest = True           Republ : Democr =     13.0 : 1.0
              prosperity = True           Republ : Democr =     13.0 : 1.0
                   taxes = True           Republ : Democr =     12.8 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

I find it interesting that 'radical' is at the top of the list. I am not surprised that 'media' is next on the list. Media plays a major role in politics, especially within recent years. It seems like there is such a jump in differences between the first two words' scores compared to the rest.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [3]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [4]:
cong_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cong_cur.fetchall())

[('websites',), ('candidate_data',), ('tweets',)]


In [5]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [11]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for rowt in results :
    # store the results in convention_data
    textt = " ".join(prepare(rowt[2],pipeline=my_pipeline))
    if textt :
        tweet_data.append([rowt[1]," ".join(prepare(rowt[2],pipeline=my_pipeline))])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [12]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)
tweet_data_sample

[['Democratic',
  'bearlier today i spoke on the house floor abt protecting health care for women and praised ppmarmonte for their work on the central coast httpstcowqgtrzt7vv'],
 ['Democratic', 'bgo tribe # rallytogether httpstco0nxutfl9l5'],
 ['Democratic',
  'bapparently trump thinks its just too easy for students overwhelmed by the crushing burden of debt to pay off student loans # trumpbudget httpstcockyqo5t0qh'],
 ['Republican',
  'bwexe2x80x99re grateful for our first responders our rescue personnel our firefighters our police and volunteers who have been working tirelessly to keep people safe provide muchneeded help while putting their own lives on the linennhttpstcoezpv0vmiz3'],
 ['Republican',
  'bletxe2x80x99s make it even greater # kag xf0x9fx87xbaxf0x9fx87xb8 httpstcoy9qozd5l2z'],
 ['Democratic',
  'bwe have about 1hr until the cavs tie up the series 22 im # allin216 repbarbaralee you scared # roadtovictory'],
 ['Democratic',
  'bcongrats to belliottsd on his new gig at sd

In [14]:

for party, tweet in tweet_data_sample :
    estimated_party = random.choices(tweet_data_sample[1],k=1)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: bearlier today i spoke on the house floor abt protecting health care for women and praised ppmarmonte for their work on the central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says ['Democratic'].

Here's our (cleaned) tweet: bgo tribe # rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says ['bgo tribe # rallytogether httpstco0nxutfl9l5'].

Here's our (cleaned) tweet: bapparently trump thinks its just too easy for students overwhelmed by the crushing burden of debt to pay off student loans # trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says ['bgo tribe # rallytogether httpstco0nxutfl9l5'].

Here's our (cleaned) tweet: bwexe2x80x99re grateful for our first responders our rescue personnel our firefighters our police and volunteers who have been working tirelessly to keep people safe provide muchneeded help while putting their own lives on the linennhttpstcoezpv0vmiz3
Actual p

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [17]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    party, tweet = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = random.choices(tweet_data_sample[1],k=1)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

TypeError: unhashable type: 'list'

In [ ]:
results

### Reflections

_Write a little about what you see in the results_ 